In [1]:
# pandas 사용해 실제 데이터 읽고 분석해보자
import pandas as pd
import numpy as np

In [5]:
# data 불러오기
df = pd.read_csv("C:/Users/SSO/python_data/data/lending-club-loan-data/loan.csv", sep=",")

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (19,47,55,112,123,124,125,128,129,130,133,139,140,141) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
# df의 크기 확인
df.shape

(2260668, 145)

In [7]:
# columns들 확인 / 유의미한 columns들 무엇일지 뽑아야 함
df.columns

Index(['id', 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv',
       'term', 'int_rate', 'installment', 'grade', 'sub_grade',
       ...
       'hardship_payoff_balance_amount', 'hardship_last_payment_amount',
       'disbursement_method', 'debt_settlement_flag',
       'debt_settlement_flag_date', 'settlement_status', 'settlement_date',
       'settlement_amount', 'settlement_percentage', 'settlement_term'],
      dtype='object', length=145)

In [8]:
# 의미있는 새로운 df 만듬
df2 = df[["loan_amnt", "loan_status", "grade", "int_rate", "term"]]

In [12]:
# 맨 뒤 다섯행의 내용 확인
df2.tail()

,loan_amnt,loan_status,grade,int_rate,term
2260663,12000,Current,C,14.08,60 months
2260664,12000,Fully Paid,E,25.82,60 months
2260665,10000,Current,B,11.99,36 months
2260666,12000,Current,D,21.45,60 months
2260667,16550,Current,D,21.45,60 months


In [13]:
# 맨 앞 5행 내용 확인
df2.head()

,loan_amnt,loan_status,grade,int_rate,term
0,2500,Current,C,13.56,36 months
1,30000,Current,D,18.94,60 months
2,5000,Current,D,17.97,36 months
3,4000,Current,D,18.94,36 months
4,30000,Current,C,16.14,60 months


In [14]:
df2["loan_status"].unique()

array(['Current', 'Fully Paid', 'Late (31-120 days)', 'In Grace Period',
       'Charged Off', 'Late (16-30 days)', 'Default',
       'Does not meet the credit policy. Status:Fully Paid',
       'Does not meet the credit policy. Status:Charged Off'],
      dtype=object)

In [15]:
df2["grade"].unique()

array(['C', 'D', 'B', 'A', 'E', 'F', 'G'], dtype=object)

In [16]:
df2["term"].unique()

array([' 36 months', ' 60 months'], dtype=object)

In [17]:
df2.shape

(2260668, 5)

In [19]:
# df2에서 하나라도 NaN 들어가있으면 (결측값 있으면) drop
df2 = df2.dropna(how="any")

In [21]:
df2.shape
# 결과가 그대로면 결측값 없다

(2260668, 5)

In [22]:
# 36개월 대출, 60개월 대출 2가지 상품 운영함 / 각각 상품을 통해 대출 이루어진 전체 총액 파악해보자
term_to_loan_amnt_dict = {}
uniq_terms = df2["term"].unique()

In [24]:
for term in uniq_terms:
    loan_amnt_sum = df2.loc[df2["term"] == term, "loan_amnt"].sum()
    term_to_loan_amnt_dict[term] = loan_amnt_sum

In [25]:
term_to_loan_amnt_dict

{' 36 months': 20517162875, ' 60 months': 13498953050}

In [26]:
term_to_loan_amnt = pd.Series(term_to_loan_amnt_dict) 
# dictionary를 Series 형태로 새로 만들기

In [28]:
term_to_loan_amnt
# index와 값 형태로 구성 된 Series

 36 months    20517162875
 60 months    13498953050
dtype: int64

In [29]:
# 대출자의 대출 상태에 따른 대출 등급 파악해보자

In [30]:
df2.head()

,loan_amnt,loan_status,grade,int_rate,term
0,2500,Current,C,13.56,36 months
1,30000,Current,D,18.94,60 months
2,5000,Current,D,17.97,36 months
3,4000,Current,D,18.94,36 months
4,30000,Current,C,16.14,60 months


In [32]:
df2["loan_status"].unique() 
# 대출 상태 불량 or not 파악 가능

array(['Current', 'Fully Paid', 'Late (31-120 days)', 'In Grace Period',
       'Charged Off', 'Late (16-30 days)', 'Default',
       'Does not meet the credit policy. Status:Fully Paid',
       'Does not meet the credit policy. Status:Charged Off'],
      dtype=object)

In [35]:
total_status_category = df2["loan_status"].unique()
bad_status_category = total_status_category[[1,2,4,5,6,8]]
# total category 중 불량 한 것 나누기

In [36]:
bad_status_category

array(['Fully Paid', 'Late (31-120 days)', 'Charged Off',
       'Late (16-30 days)', 'Default',
       'Does not meet the credit policy. Status:Charged Off'],
      dtype=object)

In [39]:
df2["bad_loan_status"] = df2["loan_status"].isin(bad_status_category)
#bad 상태인 category가 df2의 loan status에 True이면 bad_loan_status에 True / 없으면 False로

In [40]:
df2.head()

,loan_amnt,loan_status,grade,int_rate,term,bad_loan_status
0,2500,Current,C,13.56,36 months,False
1,30000,Current,D,18.94,60 months,False
2,5000,Current,D,17.97,36 months,False
3,4000,Current,D,18.94,36 months,False
4,30000,Current,C,16.14,60 months,False


In [46]:
bad_loan_status_to_grades = df2.loc[df2["bad_loan_status"] == True, "grade"].value_counts()
#bad loan_status가 true인 것 중 grade열을 가져와서 grade에 따른 value_counts 실행

In [48]:
bad_loan_status_to_grades

B    385825
C    379052
A    227834
D    201299
E     94293
F     32410
G      9320
Name: grade, dtype: int64

In [51]:
bad_loan_status_to_grades.sort_index()
# A B C 순서로 정렬

A    227834
B    385825
C    379052
D    201299
E     94293
F     32410
G      9320
Name: grade, dtype: int64

In [52]:
# 대출자가 정한 대출 총액과 대출 이자율 관의 상관관계는?

In [56]:
df2["loan_amnt"].corr(df2["int_rate"])
# 이자율과 대출 총액의 관계 / 상관관계는 그렇게 크지 않다

0.09808333533023432

In [61]:
bad_loan_status_to_grades.to_csv("bad_loan_status.csv", sep=",") 
# 위에서 만든 Series(df형태)를 csv파일로 만들어서 쓸것이야 to_csv 

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.
